# Tensors

Tensor 是 Pytorch 中最核心的数据结构，在 Pytorch 中几乎所有的形式的数据都在用 Tensor表示，模型的输入与输出格式都是 Tensor。它是一种多维数组结构，类似于 NumPy 的 `ndarray`，但具有更高的灵活性和 GPU 加速的能力。Tensor 的特点和操作在计算图（Computational Graph）中可以高效地支持梯度计算，适合自动微分（Autograd），因此在构建和训练神经网络时被广泛使用。

In [2]:
import torch
from torch import Tensor
import numpy as np

## Tensor的属性

### Tensor 的数据类型

torch 在创建 Tensor 时，`dtype`的指定只支持使用`torch.[DataType]`这样的方式去指定，而不能像numpy一样，可以直接使用字符串。

以下显示了从 `List` 创建一个新的 Tensor 时，同时指定了创建的 Tensor 的数值类型为 `int`，如果不显示的指定，那么默认是 `torch.int64`

In [3]:
t1 = torch.tensor([1, 2, 3], dtype=torch.int)
t2 = torch.tensor([1, 2, 3])
print(f"dtype of t1: {t1.dtype}")
print(f"dtype of t2: {t2.dtype}")

dtype of t1: torch.int32
dtype of t2: torch.int64


区别于 Torch，在 Numpy 中 "int" 默认代表的是 `int64`:

In [4]:
t = np.array([1, 2, 3], dtype="int")
print(f"dtype of t: {t.dtype}")

dtype of t: int64


其中`torch.dtype`表示Tensor的数据类型，常见的有下面几种不同的数据类型，和 c/c++ 中的表示是比较一致的。

浮点类型：

- `torch.float32`或`torch.float`
- `torch.float64`或`torch.double`
- `torch.float16`或`torch.half`
- `torch.bfloat16`

整数类型

- `torch.uint8`
- `torch.int8`
- `torch.int16`
- `torch.int32`
- `torch.int64`

布尔类型

- `torch.bool`

虚数类型


- `torch.complex32`: complex32 目前只是实验性的支持，后续可能会取消支持
- `torch.complex64`
- `torch.complex128`或`torch.cdouble`

虚数类型的 Tensor 在深度学习中使用的比较少，最近随机大语言模型的兴起，在相对旋转位置编码（RoPE）中有所使用。下面的代码演示了虚数 Tensor 的创建。

In [5]:
# 创建一个 Complex64 张量
real_part = torch.randn(3, 3)  # 实部
imag_part = torch.randn(3, 3)  # 虚部
complex_tensor = torch.complex(
    real_part, imag_part
)  # 构建出来的 Tensor 默认是 complex64 类型

print(complex_tensor)
print(complex_tensor.dtype)  # 输出: torch.complex64
complex_tensor = complex_tensor.to(dtype=torch.cdouble)
print(complex_tensor.dtype)  # 输出: torch.complex128

tensor([[ 0.5276+0.7824j, -0.3142+0.1675j,  0.4610+0.3706j],
        [-0.0702+0.9847j,  0.3647+0.0814j,  0.6853-1.1555j],
        [-0.3324+0.0389j,  1.4756+0.8922j, -0.2835+0.4831j]])
torch.complex64
torch.complex128


[`bfloat16`](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format)是一种和IEEE half-precision 16-bit float规定不一致的16Bit浮点数格式，它是直接对32位的IEEE 754规定的单精度float32的格式进行截取形成的，它是为机器学习系统特别定制的，它的组成是：
- 1位符号位
- 8个指数位
- 7个小数位

神经网络对指数的大小比对尾数的大小更敏感，也就是对数值的表示范围比较敏感。为了确保下溢、上溢和`NaN`的行为完全相同，`bfloat16`的指数大小与`float32`相同。`bfloat16`处理非规格化数的规则与`float32`不同，它会将它们截断为零。与通常需要特殊处理，如损失缩放的`float16`不同，`bfloat16`是训练和运行深度神经网络时`float32`的即插即用替换。

简而言之，`bfloat16`表示的数值范围更大，但是精度不如`float16`。

<div class="wy-nav-content-img">
    <img src="assets/Tensors_floating_point_formats.png" width="800px" alt="34 层的ResNet架构与 VGG19 以及线性连接的结构之间的对比">
    <p>图1: 三种浮点数格式的表示与区别</p>
</div>

另外，在 Torch 2.x 的版本中也开始实验性的支持`FP8`格式，目前支持两种类型的`FP8`格式，一种是`E3M4`一种是`E5M2`，其中`E5M2`和 IEEE 的 `fp16`的指数部分长度一致，这使得 E5M2 和 IEEE FP16 格式之间可以直接转换。`E5M2`它表达的数值范围更大，可以处理一些特殊值。

一般来说，我们会在模型推理以及训练过程的前向阶段为了保持较高的精度会使用`E3M4`的格式，而在训练的反向阶段，使用`E5M2`来有较好的数值动态范围，避免梯度消失与爆炸。

Paper: [FP8 Formats for Deep Learning](https://arxiv.org/abs/2209.05433)

<div class="wy-nav-content-img">
    <img src="assets/Tensors_FP8_format.png" width="600px" alt="二种半精度浮点数与二种 FP8 精度在存储格式上的区别">
    <p>图2: 二种半精度浮点数与二种 FP8 精度在存储格式上的区别</p>
</div>

上图展示了使用不同数值类型来表示“0.3952” 时，实际能够近似到的值。

In [6]:
import torch
from tabulate import tabulate

f32_type = torch.float32
f16_type = torch.float16
bf16_type = torch.bfloat16
e4m3_type = torch.float8_e4m3fn
e5m2_type = torch.float8_e5m2

# collect finfo for each type
table = []
for dtype in [f32_type, f16_type, bf16_type, e4m3_type, e5m2_type]:
    numbits = (
        32
        if dtype == f32_type
        else 16 if dtype == bf16_type or dtype == f16_type else 8
    )
    info = torch.finfo(dtype)
    table.append(
        [info.dtype, numbits, info.max, info.min, info.smallest_normal, info.eps]
    )

headers = ["data type", "bits", "max", "min", "smallest normal", "eps"]
print(tabulate(table, headers=headers))

data type        bits              max               min    smallest normal          eps
-------------  ------  ---------------  ----------------  -----------------  -----------
float32            32      3.40282e+38      -3.40282e+38        1.17549e-38  1.19209e-07
float16            16  65504            -65504                  6.10352e-05  0.000976562
bfloat16           16      3.38953e+38      -3.38953e+38        1.17549e-38  0.0078125
float8_e4m3fn       8    448              -448                  0.015625     0.125
float8_e5m2         8  57344            -57344                  6.10352e-05  0.25


### Tensor 的设备类型

`torch.device`表示的是Tensor的数据存储的设备，其中分为`cpu`和`cuda`

In [7]:
torch.device("cpu")

device(type='cpu')

In [8]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [9]:
torch.device(type="cuda", index=0)

device(type='cuda', index=0)

In [10]:
torch.tensor([1, 2, 3, 4, 5, 6], device=torch.device("cuda:0"))

tensor([1, 2, 3, 4, 5, 6], device='cuda:0')

### Tensor 的内存布局

Tensor 的 `layout` 属性表示Tensor内部数据存储的内部布局，目前还是一个不成熟(beta)的特性，目前支持

- torch.strided
- torch.sparse_coo

现在主要用的就是面向 dense Tensor的`torch.strided`，Tensor的 `strides` 是一个list，它代表每个dimension上两邻两个元素之间的跨度(元素个数)。

In [11]:
torch.arange(60).reshape(3, 4, 5).stride()

(20, 5, 1)

我们可以理解为 Tensor 底层的存储的是一个一维的数组，我们对于 `Tensor`的索引，全部是是通过一个下标对应的 stride 来计算出最终在一维数组上的偏移量。 这样实现的好处时，对于 `Tensor`的很多操作，它并不需要实际对 `Tensor`的内存数据进行变动。

In [12]:
t = torch.arange(12).view(3, 4)
# t_transposed 和 t 共享底层的数据
t_transposed = t.transpose(0, 1)
print(t.stride())
print(t_transposed.stride())

(4, 1)
(1, 4)


`stride()`方法除了可以直接返回一个`List`之外，还可以通过维度的索引，返回对应维度上相邻两个元素之间在底层数组上的跨度。

In [13]:
print(f"stride on dim 0: {t.stride(0)}")
print(f"stride on dim 1: {t.stride(1)}")

stride on dim 0: 4
stride on dim 1: 1


和 Numpy 不同的是，torch 中的 `stride` 以元素的个数来表示跨度，而 Numpy 则是用字节数量来表示相邻两个元素之间的跨度。

In [16]:
np.arange(60).reshape(3, 4, 5).strides

(160, 40, 8)

### Tensor属性转换

我们可以使用`to`方法来进行 Tensor 的相关属性的转换，包括：数值类型、设备类型等，接口返回的是一个新的转换后的 Tensor

In [17]:
device_cuda = torch.device("cuda")
data = torch.tensor([1])
print(data.dtype, data.device)
data = data.to(dtype=torch.float32, device=device_cuda)
print(data.dtype, data.device)

torch.int64 cpu
torch.float32 cuda:0


也可以通过 Tensor 的`dtype`方法来直接将返回新数据类型的 Tensor

In [18]:
data.int() # 将 data 转换为 int 类型
data.float() # 将 data 转换为 bool 类型
data.bool() # 将 data 转换为 bool 类型

tensor([True], device='cuda:0')

### Tensor的形状

Tensor除了具有3个标准的属性外，一旦我们创建了一个Tensor，那么它就会具有一些形状相关的属性，我们可以通过下面这些接口获取 `Tensor` 不同的尺寸相关的信息。

- `t.shape`: 返回的是一个 `torch.Size(tuple)` 类型的结果，表示每一维的维度值
- `t.size()`: 和 `t.shape` 一致
- `t.size(i)`: 返回第 `i` 个维度的值
- `t.ndim`：返回 `Tensor` 有多少维
- `t.numel()`：它是一个方法，返回 `Tensor` 内有多少个元素
- `len(t)`：返回的是 `Tensor` 在第`0`维上的维度值

In [19]:
t = torch.empty(2, 3, 4)
print(f"shape of t: {t.shape}")
print(f"size of t: {t.size()}")
print(f"size(1) of t: {t.size(1)}")
print(f"strides of t: {t.stride()}")
print(f"strides of axes{1} of t: {t.stride(1)}")
print(f"ndim of t: {t.ndim}")
print(f"numel of t: {t.numel()}")
print(f"len of t: {len(t)}")

shape of t: torch.Size([2, 3, 4])
size of t: torch.Size([2, 3, 4])
size(1) of t: 3
strides of t: (12, 4, 1)
strides of axes1 of t: 4
ndim of t: 3
numel of t: 24
len of t: 2


## Tensor的创建

在Pytorch中我们可以有多种方法来创建Tensor，常用的包括下面几种：

- 从已有的 `scalar`、`list`、`tuple`、`numpy.array` 来创建
- 用`arange`、`linspace`、`logspace`等创建一维数列 `Tensor`
- 用`ones`、`zeros`、`eye`、`full`、`empty`等来创建特别填充值的多维 `Tensor`
- 用随机数来创建指定形状的 `Tensor`

### 从现有数据来创建

我们可以使用`torch.tensor()`函数来从已有的一个 array_like 的 data 来创建一个 Tensor

In [20]:
# 从list创建
t1 = torch.tensor([1, 2, 3, 4, 5])
# 从tuple创建
t2 = torch.tensor((1, 2, 3))
# 从numpy.array创建，同时指定dtype和device
t3 = torch.tensor(np.array([1, 2, 3, 4, 5]), dtype=torch.float32, device="cuda:0")

需要注意的是，无论是从 `python` 的内置序列创建，还是从 `numpy.array` 来创建，创建出来的 `Tensor` 都是复制了原数据的内容。如果我们希望，创建的 `Tensor` 不额外分配存储空间，而是和之前的 `numpy.array` 共享存储，那么可以使用`as_tensor`方法

In [23]:
arr = np.array([1, 2, 3, 4, 5])
print(f"原始的array: ", arr)
t = torch.as_tensor(arr)
# 对于Tensor的数据改动，也会影响在ndarray上
t[0] = 6
print(f"修改后的arr: ", arr)

原始的array:  [1 2 3 4 5]
修改后的arr:  [6 2 3 4 5]


不过使用`as_tensor`后，能共享底层存储的，前提是，`as_tensor` 方法中指定的`dtype`和`device`和原 `ndarry` 是一致的。由于 `numpy` 不支持 CUDA，所以这样只能创建 cpu 上的 Tensor

In [24]:
arr = np.array([1, 2, 3, 4, 5]) # 原 arr 的数值类型是 int64
t = torch.as_tensor(arr, dtype=torch.float32)  # 这种情况下，并不会共享底层存储
t[0] = 6
print(arr)

[1 2 3 4 5]


当我们使用 List 来创建 Tensor 时，需要注意对于数据类型的转换处理：对于 int 类型，NDArray 和 Tensor 都会转换为 `int64`，对于浮点数 float，Tensor 默认转换为`float32`，而 NDArray 则默认转换为 `float64`

In [ ]:
il = [1, 2, 3, 4, 5]
print(f"ndarray的默认整数类型为:{np.array(il).dtype}")
print(f"tensor的默认整数类型为: {torch.tensor(il).dtype}")

fl = [1.0, 2.0, 3.0, 4.0, 5.0] # List[float]
print(f"ndarray的默认整数类型为:{np.array(fl).dtype}")
print(f"tensor的默认整数类型为: {torch.tensor(fl).dtype}")

ndarray的默认整数类型为:int64
tensor的默认整数类型为: torch.int64
ndarray的默认整数类型为:float64
tensor的默认整数类型为: torch.float32


从另外一个 Tensor 来创建 Tensor，无论 b 是否指新新的 `dtype` 和 `device`，b 都不和 a 共享数据，根据 Warning 提示，我们知道这种用法目标在 Pytorch 中是不推荐的，如果要复制 Tensor，可以直接用 `clone` 方法。

In [29]:
a = torch.tensor([1, 2, 3])
b = torch.tensor(a, dtype=torch.float, device="cuda:0")

/tmp/ipykernel_4590/1755634830.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.tensor(a, dtype=torch.float, device="cuda:0")


In [30]:
b = a.clone()
b = b.to(device="cuda:0")
print(b)

tensor([1, 2, 3], device='cuda:0')


### torch.Tensor()

请注意 `torch.Tensor`和 `torch.tensor`的不同。 `torch.Tensor` 实际上是`torch.FloatTensor`，用它来创建新的Tensor时，实际调用的是构造函数，它会默认以`torch.float32`来作为`dtype`。而`torch.tensor`会根据`data`的类型自动推断。

In [31]:
l = [1, 2, 3, 4, 5]
print(torch.Tensor(l).dtype)
print(torch.tensor(l).dtype)

torch.float32
torch.int64


### 创建特别填充值的Tensor

#### `torch.arange`

`torch.arange(start=0, end, step=1)` 用于创建一个区间范围的 Tensor

In [32]:
print(torch.arange(5))
print(torch.arange(1, 5))
print(torch.arange(1, 20, 3))

tensor([0, 1, 2, 3, 4])
tensor([1, 2, 3, 4])
tensor([ 1,  4,  7, 10, 13, 16, 19])


如果 `start`、`end` 以及 `step` 中有浮点数，则创建出来的是 `FloatTensor`

In [33]:
torch.arange(1, 3.5, 0.5)

tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000])

注意上面是没有包括 3.5 那个点的

#### `torch.linspace`

`torch.linspace`与`torch.arange`有点类似，都指定一个起点，一个终点，和一个步长。但`linspace`里步长最终指定了生成的一维 Tensor 中元素的个数

```python
def linspace(start:float ,end:float ,steps:int) -> Tensor:
    pass
```

另外需要注意的是`torch.linspace`生成的一定是一个浮点数的Tensor，而且和`torch.arange`不同的是：`linspace`生成的Tensor是包括末点值的（inclusive）

In [34]:
torch.linspace(3, 10, 5)

tensor([ 3.0000,  4.7500,  6.5000,  8.2500, 10.0000])

#### `torch.logspace`

`torch.logspace`和`torch.linspace`行为类似，区别在于`logspace`生成的序列的范围的起始与终点是一个以`base`为底，`start`和`end`为指数的数字。

```python
def logspace(start:float ,end:float ,steps:int, base=10.0) -> Tensor:
    pass
```

#### `torch.ones`、`torch.zeros`、`torch.emtpy`

它们三个都是用于创建一个指定 `size` 的Tensor，分别以`1`、`0`和未初始化的值来填充创建好的 `Tensor`。它们三个返回的都是 `FloatTensor`

In [35]:
print(torch.ones((2, 2)))
print(torch.zeros((3, 4)))
print(torch.empty((3, 3)))

tensor([[1., 1.],
        [1., 1.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[2.5159e-33, 0.0000e+00, 0.0000e+00],
        [1.4013e-45, 0.0000e+00, 0.0000e+00],
        [4.6243e-44, 0.0000e+00, 2.5176e-33]])


`torch.ones/zeros/empty`支持`torch.ones(d1,d2,...)`这种调用方法，而 `numpy` 则不支持。

#### `torch.eye`

`torch.eye` 返回的是一个 2D 的对角线为 `1`，其他值都为`0` 的 `FloatTensor`

In [37]:
torch.eye(4)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

#### `torch.full`

`torch.full`返回的是一个指定`size`和填充值的Tensor，Tensor的 `dtype` 是由填充值的类型来推导的。

```python
def full(size, fill_value) -> Tensor:
  '''
  Args:
    size(int...): a list ,tuple or torch.Size
    fill_vale(Scalar)
  '''
  pass
```

In [38]:
torch.full((2, 3), 1.0)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

#### `torch.diag`

如果输入的是一个 `1d` 的 Tensor，则返回的是一个 `2d` 的对角矩阵，其对角线上的元素为传入的 `Tensor`，也可以通过`diagonal`来指定对角线元素的轴偏值。

In [34]:
torch.diag(torch.tensor([1, 2, 3]))

tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])

In [ ]:
torch.diag(torch.tensor([1, 2, 3]), diagonal=-1)

tensor([[0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 2, 0, 0],
        [0, 0, 3, 0]])

传入 `2d` 的 `Tensor`，则返回 `Tensor` 的对角线上的元素，返回的是一个 `1d` 的 `Tensor`

In [39]:
torch.diag(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

tensor([1, 5, 9])

In [40]:
torch.diag(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), diagonal=1)

tensor([2, 6])

### 使用随机数来创建 `Tensor`

#### `torch.normal`

`torch.normal`返回一个正态分布产生在的随机数填充的Tensor，它一共有4种参数传递方式

* 第一种是: `torch.norm(mean, std)` 其中 `mean` 和 `std` 都是一个 Tensor，生成的 `Tensor` 的形状和 `mean` 和`std` 的形状是一致的，其中每个元素都是通过对应位置的 `mean` 和 `std` 形成的正态分布来随机产生的。
* 第二种是: `normal(mean=0.0, std, *, out=None)`。这种参数传递用法，与上面的区别就是 `mean` 变成一个 `scalar`，那么说明每个元素来共享一个 `mean` 值。在这种情况下。
* 第三种是: `normal(mean, std=1.0, *, out=None)`。这种情况和第 2 种情况，恰恰相反了，`std` 变成了每个元素共享的。
* 第四种是: `normal(mean, std, size, *, out=None)`。这种情况下，所有的元素都共享 `mean` 和 `std`，最终 `Tensor` 的形状是由 `size` 来决定的。

In [41]:
mean = torch.randn(3, 4)
std = torch.rand((3, 4))
torch.normal(mean, std)

tensor([[-1.4458, -0.8187,  2.4465,  2.0837],
        [ 1.6485,  0.4339, -0.8377, -0.0096],
        [ 0.2849, -0.7824, -0.7216,  0.4097]])

In [42]:
torch.normal(1.0, std)

tensor([[ 1.0569,  0.8937,  1.3969,  1.0340],
        [ 0.7369,  1.0320, -0.1283,  1.2739],
        [ 0.8920,  0.8142, -0.0419,  2.0956]])

In [43]:
torch.normal(mean, 0.5)

tensor([[-0.5882, -0.3526,  0.6500,  1.8875],
        [ 0.9137,  0.3099, -0.3028,  0.6313],
        [-0.0521, -0.6022, -0.5374, -0.8275]])

In [44]:
torch.normal(0, 1, (3, 4))

tensor([[ 0.2031, -2.4626,  0.4687,  0.1620],
        [-1.9686,  0.6622,  1.1888,  0.3248],
        [-0.0875,  1.9568, -0.1608, -2.3748]])

#### `torch.rand`、`torch.randn`

* `rand`直接生成指定形状的 `Tensor`，其中每个元素都是由`[0,1)`均匀分布来随机产生。
* `randn`直接生成指定形状的 `Tensor`，其中每个元素都是由标准正态分布来随机产生。

In [45]:
torch.rand(3, 4)  # 或者 torch.randn((3,4))

tensor([[0.0958, 0.9279, 0.8892, 0.1625],
        [0.6920, 0.3723, 0.9931, 0.8634],
        [0.4582, 0.4525, 0.1676, 0.4868]])

In [46]:
torch.randn(3, 4)  # 或者 torch.randn((3,4))

tensor([[-0.5794,  0.4422,  1.1399, -0.0520],
        [-0.8622, -0.9455, -1.0510, -0.3753],
        [-0.6126,  1.3127,  1.1930,  1.3659]])

#### `torch.randint`

```python
randint(low=0, high, size, **kwargs)
```
产生一个由 `[low,high)` 区间均匀分布随机数填充的 `LongTensor`

In [47]:
torch.randint(1, 10, (3, 4))

tensor([[5, 8, 3, 7],
        [1, 1, 1, 3],
        [2, 7, 1, 6]])

#### `torch.randperm`

生成一个随机全排列的一维的 `LongTensor`

In [48]:
torch.randperm(12)

tensor([ 3,  1, 10,  9, 11,  6,  7,  0,  8,  5,  2,  4])

### 使用`xx_like`系列创建相同形态的Tensor

除了shape保持一致外，`dtype`、`layout`、`device`等，若无特别指定，则也与源Tensor保持一致。

```python
torch.zeros_like(input, **kwargs) # 返回与input相同size的零矩阵
torch.ones_like(input, **kwargs) #返回与input相同size的单位矩阵
torch.full_like(input, fill_value, **kwargs) #返回与input相同size，单位值为fill_value的矩阵
torch.empty_like(input, **kwargs) # 返回与input相同size,并被未初始化的数值填充的tensor
torch.rand_like(input, dtype=None, **kwargs) #返回与input相同size的tensor, 填充均匀分布的随机数值
torch.randint_like(input, low=0, high, dtype=None, **kwargs) #返回与input相同size的tensor, 填充[low, high)均匀分布的随机数值
torch.randn_like(input, dtype=None, **kwargs) #返回与input相同size的tensor, 填充标准正态分布的随机数值
```

In [50]:
src = torch.randn(4, 5)

In [51]:
torch.zeros_like(src)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [52]:
torch.ones_like(src, dtype=torch.int)

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.int32)

In [53]:
torch.empty_like(src, device="cuda:0")

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')

In [54]:
# 这里即使full_value是int类型，但生成的Tensor，依然是用的src的dtype
torch.full_like(src, 42)

tensor([[42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42.]])

In [55]:
torch.rand_like(src)

tensor([[0.5867, 0.2891, 0.5580, 0.1204, 0.0054],
        [0.1634, 0.9599, 0.0095, 0.0271, 0.5853],
        [0.5226, 0.4830, 0.8212, 0.2810, 0.7092],
        [0.5244, 0.4347, 0.5706, 0.8941, 0.2371]])

In [56]:
torch.randn_like(src)

tensor([[ 0.3273, -1.6254,  0.2422, -0.0030, -1.1903],
        [-0.9231,  1.0337,  0.6556,  0.5820,  0.3665],
        [-0.0355,  1.1695,  1.5428,  0.4138,  0.6371],
        [ 0.4081, -1.8035, -0.9763, -0.6170,  0.4965]])

In [57]:
torch.randint_like(src, 1, 10)

tensor([[8., 9., 2., 9., 3.],
        [8., 8., 3., 3., 9.],
        [3., 1., 4., 3., 9.],
        [8., 6., 8., 7., 6.]])

## Tensor的操作

Pytorch中的Tensor大约支持100种以上的操作，其中包括了数学运算、线性代数、矩阵操作（转置、索引、切片等），这些操作都可以跑在CPU或GPU上，这也是 Pytorch Tensor的强大之处。

![](../images/tensor_operatrions.png)

我们可以通过这个[页面](https://pytorch.org/docs/stable/torch.html)，来对Tensor支持的所有操作做个大概的了解。

### 索引访值

#### 基础索引

我们可以像访问Numpy.ndarray一样，对torch.Tensor进行各种下标索引与范围切片。

In [54]:
t = torch.arange(12).reshape(3, 4)
print(f"t: {t}")
print(f"取t的第2行的所有元素: {t[1]}")
print(f"取t的最后一列的所有元素: {t[:, -1]}")
print(f"取t的第2列到最后一列的所有元素: {t[:, 2:]}")
print(f"取t的位置(2,3)上的元素: {t[2, 3]}")

t: tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
取t的第2行的所有元素: tensor([4, 5, 6, 7])
取t的最后一列的所有元素: tensor([ 3,  7, 11])
取t的第2列到最后一列的所有元素: tensor([[ 2,  3],
        [ 6,  7],
        [10, 11]])
取t的位置(2,3)上的元素: 11


**单一元素的Tensor**

当我们通过索引访问Tensor的单一元素时，得到的实际是一个`Tensor`类型的对象，它并不是python中的内置数据类型，我们可以通过Tensor的`item()`方法来获取python对象的标量。

In [55]:
type(t[2, 3])

torch.Tensor

In [56]:
t[2, 3].shape

torch.Size([])

In [57]:
type(t[2, 3].item())

int

注意对如果某个维度上我们只取一行/列数据，那么有两种方式，这两种方式得到的结果的 Shape 会不一样

In [58]:
t1 = t[:, 2:3]
t2 = t[:, 2]
print(t1)  # 还是一个二维的Tensor
print(t2)  # 一维的 Tensor

tensor([[ 2],
        [ 6],
        [10]])
tensor([ 2,  6, 10])


#### 高级索引

Tensor 的高级索引，支持我们直接用一个 Long 型的 Tensor 作为索引来取原 Tensor 中的元素。

In [59]:
t = torch.randn(8, 10)
# indices 的所有元素都代表 t 的 dim=0 的下标
indices = torch.randint(0, 8, (3, 2))
t[indices].shape

torch.Size([3, 2, 10])

#### torch.gather

torch.gather 往往用于我们希望依次在输入 Tensor 的某个维度上取出其中一些索引的值。比如下面的 topK 的结果中，我们希望根据返回的 indices 取得对应的元素，也就是 values，这里可以用 gather

In [60]:
t = torch.randn(3, 5)
values, indices = torch.topk(t, k=2, dim=1)
print(t)
print(values)
print(indices)

tensor([[-2.2757, -0.3290,  0.7861,  1.6445, -0.3443],
        [-1.5337,  0.8690,  2.1442,  1.0928, -0.7280],
        [ 0.4658, -0.8848,  1.4970,  0.2225, -0.6478]])
tensor([[1.6445, 0.7861],
        [2.1442, 1.0928],
        [1.4970, 0.4658]])
tensor([[3, 2],
        [2, 3],
        [2, 0]])


In [61]:
values == torch.gather(t, dim=1, index=indices)

tensor([[True, True],
        [True, True],
        [True, True]])

### 组合与分片

#### torch.cat

```
cat(tensors, dim=0) -> Tensor
```

`torch.cat`将给定义的tensor的序列(tensors)，按给定义的维度上合并起来，这就要求，这些tensor，除了合并的维度，其他的维度必须一致。

In [62]:
t1 = torch.randn(2, 3)
t2 = torch.randn(3, 3)
torch.cat([t1, t2], dim=0)

tensor([[ 0.8229, -0.4819, -0.0436],
        [ 1.1926,  0.3573,  0.6702],
        [-1.3167,  0.1392, -0.3058],
        [ 1.3075,  0.8602, -0.2774],
        [-0.4457,  1.1762, -0.4505]])

#### torch.stack

`torch.stack`和`torch.cat`接口用法一致，但它并不是在原有的维度上拼接，而是直接扩展一个新的维度。

这就要求，序列中的tensor在维度上必须一致。

In [63]:
t1 = torch.randn(2, 3)
t2 = torch.randn(2, 3)
torch.stack([t1, t2], dim=0)

tensor([[[-0.1507, -0.6556, -0.5804],
         [-0.9632, -0.4136, -0.6468]],

        [[ 0.0827, -0.3218,  0.6109],
         [ 1.9358, -0.6337,  0.6983]]])

#### torch.split

```python
split(tensor, split_size_or_sections, dim=0)
```

`split`将tensor按指定的维度，分拆为多个Tensor的元组，拆分的块chunk的大小是splite_size指定的。可能出现不能整分的情况，这时候最后一块大小一般小于splite_size

split出来的Tensor是原tensor的一个view

In [64]:
a = torch.arange(10).view(5, 2)
a

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [65]:
torch.split(a, 2)

(tensor([[0, 1],
         [2, 3]]),
 tensor([[4, 5],
         [6, 7]]),
 tensor([[8, 9]]))

`split_size_or_sections`也可能是一个list(int)，这时候，它的每个元素，代表每个chunk的大小

In [66]:
a1, a2, a3 = torch.split(a, (1, 3, 1))
a1, a2, a3

(tensor([[0, 1]]),
 tensor([[2, 3],
         [4, 5],
         [6, 7]]),
 tensor([[8, 9]]))

In [67]:
# 切分出来的tensor和原tensor是共享存储的
a1[0, 0] = 42
a

tensor([[42,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9]])

#### torch.chunk

```python
chunk(input, chunks, dim=0) -> List of Tensors
```
`chunk`和`split`功能类似，不同在于，chunk的第二的参数，直接指定的是chunk的数量，最后一个chunk的数量可能会少一些。也有可能`axis[dim]<chunks`，那么就直接切分为`axis[dim]`个。

切分出来的这些Tensor和原Tensor都是共享底层存储的，也就是说每个chunk都是原Tensor的一个view。

In [68]:
print(a.shape)
len(a.chunk(3, dim=1))

torch.Size([5, 2])


2

### 变换操作

#### torch.reshape

```python
reshape(input, shape) -> Tensor
```
`reshape`返回一个和原Tensor具有相同数据，相同数量的Tensor，只是shape不一致。

#### torch.view

torch.view vs. torch.reshape

`reshape`可以用在`compact`或`non-compact`的tensor上，而`view`只能用在`compact`的tensor上。`reshape`如果作用于`non-compact`的tensor上，则会产生一个copy

torch.view has existed for a long time. It will return a tensor with the new shape. The returned tensor will share the underling data with the original tensor. See the documentation here.

On the other hand, it seems that torch.reshape has been introduced recently in version 0.4. According to the document, this method will

> Returns a tensor with the same data and number of elements as input, but with the specified shape. When possible, the returned tensor will be a view of input. Otherwise, it will be a copy. Contiguous inputs and inputs with compatible strides can be reshaped without copying, but you should not depend on the copying vs. viewing behavior.

It means that torch.reshape may return a copy or a view of the original tensor. You can not count on that to return a view or a copy. According to the developer:

> if you need a copy use clone() if you need the same storage use view(). The semantics of reshape() are that it may or may not share the storage and you don't know beforehand.

Another difference is that reshape() can operate on both contiguous and non-contiguous tensor while view() can only operate on contiguous tensor. Also see here about the meaning of contiguous.

#### torch.transpose

```python
transpose(input, dim0, dim1) -> Tensor
```
转置input的指定的2个维度，返回的Tensor和原来的Tensor共享存储

In [69]:
x = torch.rand(2, 3, 4)
x

tensor([[[0.5260, 0.9209, 0.5547, 0.3062],
         [0.3510, 0.7916, 0.8831, 0.1895],
         [0.6874, 0.8094, 0.3321, 0.4644]],

        [[0.9875, 0.1983, 0.8904, 0.6557],
         [0.8619, 0.5538, 0.3221, 0.7695],
         [0.5407, 0.2839, 0.0598, 0.8637]]])

In [70]:
y = torch.transpose(x, 0, 2)

In [71]:
print(x.stride())
print(y.stride())

(12, 4, 1)
(1, 4, 12)


#### torch.permute

In [72]:
torch.permute(x, (2, 0, 1)).shape

torch.Size([4, 2, 3])

#### squeeze和unsqueeze

squeeze在指定的维度上添加一维，而unsqueeze则在指定的维度上去掉`size=1`的维度，如果对应维度上的size不等于1，则不做任何操作

In [73]:
x = torch.randn(2, 3)
y = x.unsqueeze(dim=1).unsqueeze(dim=0)
print(f"x shape: {x.shape}, \ny (x.unsqueeze) shape: {y.shape}")
print("unsqueeze shape", y.squeeze(dim=0).shape)

x shape: torch.Size([2, 3]), 
y (x.unsqueeze) shape: torch.Size([1, 2, 1, 3])
unsqueeze shape torch.Size([2, 1, 3])


#### contiguous

There are a few operations on Tensors in PyTorch that do not change the contents of a tensor, but change the way the data is organized. These operations include:

`narrow()`, `view()`, `expand()` and `transpose()`

For example: when you call transpose(), PyTorch doesn't generate a new tensor with a new layout, it just modifies meta information in the Tensor object so that the offset and stride describe the desired new shape. In this example, the transposed tensor and original tensor share the same memory:

In [74]:
x = torch.randn(3, 2)
y = torch.transpose(x, 0, 1)
x[0, 0] = 42
print(y[0, 0])

tensor(42.)


This is where the concept of contiguous comes in. In the example above, x is contiguous but y is not because its memory layout is different to that of a tensor of same shape made from scratch. Note that the word "contiguous" is a bit misleading because it's not that the content of the tensor is spread out around disconnected blocks of memory. Here bytes are still allocated in one block of memory but the order of the elements is different!

When you call contiguous(), it actually makes a copy of the tensor such that the order of its elements in memory is the same as if it had been created from scratch with the same data.

Normally you don't need to worry about this. You're generally safe to assume everything will work, and wait until you get a RuntimeError: input is not contiguous where PyTorch expects a contiguous tensor to add a call to contiguous().

### 降维操作

#### torch.mean

```python
'''
Args:
  input (Tensor): the input tensor.
  dim (int or tuple of ints): the dimension or dimensions to reduce.
  keepdim (bool): whether the output tensor has :attr:`dim` retained or not.
'''
mean(input, dim, keepdim=False, *, out=None) -> Tensor
```

对input沿着`dim`的维度求均值，这样的话，指定的那个维度就会被压缩掉，如果指定了`keepdim=True`的话，那个维度会保留，值为1

In [75]:
t = torch.randn(5, 6)
t

tensor([[-0.2039,  1.2231, -0.3430,  0.3751, -0.7528, -1.1104],
        [ 0.4561, -0.9779, -0.3400,  0.6377,  0.8848, -1.5154],
        [ 1.4089, -1.1560, -0.8236, -0.5375,  1.2653, -0.3280],
        [ 0.3497,  0.5302, -1.1714, -0.9095, -1.1539, -0.7436],
        [-0.0800, -0.4530,  0.6272,  0.7534, -0.2480,  1.8906]])

In [76]:
# 按列的方向(dim=0)将整个Tenoor压缩成为1维的
torch.mean(t, dim=0)

tensor([ 0.3862, -0.1667, -0.4102,  0.0638, -0.0009, -0.3613])

In [77]:
torch.mean(t, dim=1, keepdim=True)

tensor([[-0.1353],
        [-0.1424],
        [-0.0285],
        [-0.5164],
        [ 0.4151]])

对于高维Tensor，我们还可以同时对多个维度进行Reduce，求其均值。

In [78]:
t = torch.randn(2, 3, 4)
t

tensor([[[ 0.8714, -0.5348, -0.1684, -0.7420],
         [ 1.6161,  0.9631,  1.7041, -0.0590],
         [-0.9798,  0.8068,  0.3488, -0.2003]],

        [[ 0.4771,  1.4805,  0.2393, -0.7670],
         [ 1.2054, -0.4611, -0.3424, -0.6664],
         [ 1.7911, -0.9301, -0.4897, -1.6210]]])

In [79]:
# 等价于reduce第0维，得到一个3x4的Tensor后，再reduce第1维，得到(3,)的Vector
torch.mean(t, dim=(0, 2))

tensor([ 0.1070,  0.4950, -0.1593])

In [80]:
t.mean(0).mean(1)

tensor([ 0.1070,  0.4950, -0.1593])

#### torch.sum

`torch.sum`是一个和`torch.mean`用法上很像的操作，只是`sum`的reduce op变成了求和，而不是求均值。

In [81]:
torch.sum(t, dim=(0, 2))

tensor([ 0.8562,  3.9598, -1.2741])

#### torch.argmax

In [82]:
x = torch.rand((2, 3))
print("x:", x)
print("Argmax:", x.argmax(dim=1))

x: tensor([[0.3633, 0.4314, 0.5305],
        [0.6689, 0.1378, 0.3848]])
Argmax: tensor([2, 0])


#### torch.maxmimu

相同 Shape 的 Tensor 和 Tensor 按元素逐个比大小，保留最大的

In [83]:
def relu(x):
    return torch.maximum(x, torch.tensor(0))


x = torch.randn((2, 3))
print(f"x:\n\t{x} \nrelu(x):\n\t{relu(x)}")

x:
	tensor([[-0.1348, -0.6546, -0.5181],
        [ 0.2910,  0.7669, -1.1600]]) 
relu(x):
	tensor([[0.0000, 0.0000, 0.0000],
        [0.2910, 0.7669, 0.0000]])


### 排序

#### torch.sort

```python
sort(input, dim=-1, descending=False, *, out=None) -> (Tensor, LongTensor)
```
`sort`对input按给定义的dim进行升序排列，返回排列后的Tensor的同时，也返回一个对应的下标的重排后的Tensor

dim的默认值是Tensor的最后一维

In [84]:
a = torch.rand(2, 3)
print(a)
values, indices = torch.sort(a, dim=1, descending=True)
print(values)
print(indices)

tensor([[0.5611, 0.0464, 0.8813],
        [0.9126, 0.0398, 0.2651]])
tensor([[0.8813, 0.5611, 0.0464],
        [0.9126, 0.2651, 0.0398]])
tensor([[2, 0, 1],
        [0, 2, 1]])


#### torch.topk

```python
topk(input, k, dim=None, largest=True, sorted=True, *, out=None) -> (Tensor, LongTensor)
```
`topk`返回input中指定维度上，最大的k个元素，以及对应的索引。

In [85]:
a = torch.randn(5)
a

tensor([ 0.3229, -0.3369, -0.2835,  0.0023, -1.6770])

In [86]:
values, indices = torch.topk(a, 3)
print(values)
print(indices)

tensor([ 0.3229,  0.0023, -0.2835])
tensor([0, 3, 2])


#### torch.kthvalue

```python
kthvalue(input, k, dim=None, keepdim=False, *, out=None) -> (Tensor, LongTensor)
```
`kthvalue`计算输出Tensor的指定维度上第`k`小的元素以及下标。如果dim没有指定，则默认为Tensor的最后一维。

In [87]:
a = torch.randn(4, 3)
a

tensor([[ 0.1169, -1.1037, -0.8417],
        [ 0.0679, -1.0995,  0.8263],
        [-0.0570,  1.6181, -0.2661],
        [-0.9555, -0.3374, -0.2083]])

In [88]:
torch.kthvalue(a, 2, dim=0)

torch.return_types.kthvalue(
values=tensor([-0.0570, -1.0995, -0.2661]),
indices=tensor([2, 1, 2]))

### 原地操作(in-place)

pytorch的Tensor支持了很多原地操作，它们的特点就是在方法末尾以`_`结束

In [89]:
t1 = torch.ones(2, 3)
print(f"t1 = {t1}")
t1.add_(2)
print(f"after plus 2: t1 = {t1}")

t1 = tensor([[1., 1., 1.],
        [1., 1., 1.]])
after plus 2: t1 = tensor([[3., 3., 3.],
        [3., 3., 3.]])


### 转换为其他数据类型

我们可以调用`numpy`接口,返回一个numpy.ndarray的对象，可以调用`tolist`接口，返回一个list的对象

In [90]:
t = torch.tensor([1, 2, 3, 4, 5, 6])

In [91]:
# 返回的ndarray还是和t是共享存储的
t.numpy()

array([1, 2, 3, 4, 5, 6])

In [92]:
t.reshape(2, 3).tolist()

[[1, 2, 3], [4, 5, 6]]

### repeat和repeat_interleave

In [93]:
a = torch.arange(6).reshape((2, 3))
a

tensor([[0, 1, 2],
        [3, 4, 5]])

`repeat(d0, d1, d2)` 将对应的维度复制多份，如果之前没有对应的维度，则可以当作原来维度为1，处理。

In [94]:
a.repeat((2, 1, 2))

tensor([[[0, 1, 2, 0, 1, 2],
         [3, 4, 5, 3, 4, 5]],

        [[0, 1, 2, 0, 1, 2],
         [3, 4, 5, 3, 4, 5]]])

`repeat_interleave(n, dim)` 在对应的维度上进行复制，但复制的方式不是`[a b c a b c ]`这种，而是`[a a b b c c]`

In [95]:
a.repeat_interleave(2, dim=0)

tensor([[0, 1, 2],
        [0, 1, 2],
        [3, 4, 5],
        [3, 4, 5]])

## 爱因斯坦标识

In [96]:
from einops import rearrange, reduce, repeat

### 实现 Transpose 和 Permute 的功能

In [97]:
x = torch.randn(2, 3, 8, 8)

# Transose
torch.allclose(rearrange(x, "b c h w->b h w c"), x.permute((0, 2, 3, 1)))

True

### 一步实现 Transpose + Reshape

In [98]:
# 一步完成 Transpose + Reshape
torch.allclose(
    rearrange(x, "b c h w -> (b h w) c"), x.permute(0, 2, 3, 1).reshape(-1, x.size(1))
)

True

### 实现维度拆分

In [99]:
x = torch.randn(2, 3, 64)

torch.allclose(rearrange(x, "b c (h w) -> b c h w", h=8), x.reshape(2, 3, 8, 8))

True

### 实现 Image2Patch 的功能

将 二维图像转换为 $B\times N \times D$ 的序列 Patches 的形式。

In [100]:
image = torch.randn(2, 3, 256, 256)

patches = rearrange(image, "b c (h1 ph) (w1 pw) -> b (h1 w1) (ph pw c)", ph=8, pw=8)
print(patches.shape)

torch.Size([2, 1024, 192])


### Reduce 操作

In [101]:
x = torch.randn(8, 10)

# mean
x_mean = reduce(x, "b d -> b", reduction="mean")
# sum
x_sum = reduce(x, "b d -> 1 d", reduction="sum")

x = torch.randn(2, 3, 4)
x_max = reduce(x, "b n d -> d", reduction="max")

### 扩维与复制

In [102]:
x = torch.randn(5, 5)
rearrange(x, "i j -> 1 i j").shape

torch.Size([1, 5, 5])

In [103]:
x = torch.randn(1, 5, 5)
repeat(x, "1 i j -> 3 i (5 j)").shape

torch.Size([3, 5, 25])